# Y\*Int Social Media Analysis
#### Ester Zhao

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [30]:
url = 'https://raw.githubusercontent.com/data-challengers/DC5/main/DC5-Data/YInt%20Social%20Media%20Data/YInt.csv'
df = pd.read_csv(url, parse_dates=[0], infer_datetime_format=True)
df.head()

['Weston' 'Southton' 'Broadview' 'West Parton' 'Old Town'
 'Terrapin Springs' 'Downtown' 'Southwest' 'Scenic Vista' 'East Parton'
 'Cheddarford' 'Palace Hills' 'Safe Town' 'Easton' 'Chapparal' 'Northwest'
 'Oak Willow' 'Pepper Mill' 'Wilson Forest' 'UNKNOWN'
 '<Location with-held due to contract>']


In [36]:
# Create string parser for df
def find_string_in_messages(df, string_to_find):
    subset_df = df[df['message'].str.contains(string_to_find, na=False)]
    subset_df.reset_index()
    return subset_df

In [37]:
# Get list of neighborhoods being used in Yint dataset
neighborhoods = df['location'].unique()
print('St Himark Neighborhoods: ', len(neighborhoods), '\n',neighborhoods)

St Himark Neighborhoods:  21 
 ['Weston' 'Southton' 'Broadview' 'West Parton' 'Old Town'
 'Terrapin Springs' 'Downtown' 'Southwest' 'Scenic Vista' 'East Parton'
 'Cheddarford' 'Palace Hills' 'Safe Town' 'Easton' 'Chapparal' 'Northwest'
 'Oak Willow' 'Pepper Mill' 'Wilson Forest' 'UNKNOWN'
 '<Location with-held due to contract>']


In [40]:
# Get subset of messages sent from outside neighborhoods
other_neighborhoods = df[df['location'].isin(['UNKNOWN', '<Location with-held due to contract>'])]
non_neighborhood_users = other_neighborhoods['account'].unique()
print(non_neighborhood_users)

['CelebrityBrattyMatty' 'VeryPopularSingerLalisaManoban' 'CuriousGCandy'
 'SmartBBread' 'DarkCandyWm_Wood' 'HimarkBand' 'GertieHorse74'
 'FleetTIcecream' 'FastPBread72' 'FastBreadSteven_Chappell'
 'FrankieFish96' 'SmartCarRoy' 'Simpson2002' 'WileyIcecreamBenny_Krick'
 'BusyTurnerPlate' 'RobertsonCat14' 'FuriousJLight' 'FearsJohnBird'
 'FearsKenneth_BarnettOctopus' 'CandidFCouch' 'LazyWBread'
 'FastLightWilliam' 'DanielFish19' 'SAttentativeT' 'CuriousKatherynBanana'
 'SmartJCouch' 'WileyMccardleCar50' 'RayDog1978' 'CandidUzielCouch1967'
 'WileyLightScottie_Garcia' 'Jean_Goss86' 'Regina51F' 'DionHorse2011'
 'ObnoxiousBLight1983' 'LovesElizabeth_MoralesLion' 'FastLLight98'
 'GuapoBananaAllen' 'SmartFBowl12' 'RunsThompsonMouse' 'FleetLightMichael'
 'FastBCar' 'WileyPlateLupe_Cole' 'CuriousWBanana73' 'City EOC'
 'DarkCandyChristopher_Iverson' 'Gordon1985' 'FigueroaOctopus1959'
 'GuapoKCar' 'CuriousPlateJohn_Paine' 'Oskr' 'Wiretap Phone Company'
 'Pinne' 'Teresa97A' 'AttentativeDBanana15' 'B

In [42]:
rumble_df = find_string_in_messages(df, "rumble")
print(rumble_df)

rumble_neighborhoods = rumble_df['location'].unique()
print(rumble_neighborhoods)

                     time          location                      account  \
4     2020-04-06 00:07:00       West Parton    CuriousPlateBobbie_Mallon   
9     2020-04-06 00:16:00         Southwest                Grant1953Rapp   
20    2020-04-06 00:50:00            Weston                FastBBanana40   
24    2020-04-06 01:01:00       West Parton      Marisela_KozmaMouse1991   
26    2020-04-06 01:08:00            Weston             Manuel_OrrLion64   
...                   ...               ...                          ...   
40833 2020-04-10 09:36:00       West Parton                     LazyHCar   
41413 2020-04-10 11:01:00          Southton               ESmartLuescher   
41757 2020-04-10 11:38:00         Southwest               Christine1963M   
41770 2020-04-10 11:40:00          Downtown              SmartJHouse1985   
41858 2020-04-10 11:49:00  Terrapin Springs  AttentativeFrancisBread2011   

                                                 message  
4      You obviously need to

In [ ]:
# Plot frequency of messaging from neighborhoods over time
